In [2]:
%pip install pdfplumber

   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   --- ------------------------------------ 0.5/5.6 MB 4.2 MB/s eta 0:00:02
   ------------------------------- -------- 4.5/5.6 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 5.6/5.6 MB 13.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 21.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   ---------------------------------------- 3.0/3.0 MB 15.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 15.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pdfplumber
import re
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tree import Tree

# Ensure NLTK corpora are downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

def extract_name(text):
    """Extract name using Named Entity Recognition."""
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        tokens = nltk.word_tokenize(sentence)
        tags = nltk.pos_tag(tokens)
        chunks = ne_chunk(tags)
        for chunk in chunks:
            if isinstance(chunk, Tree) and chunk.label() == 'PERSON':
                return " ".join(c[0] for c in chunk)
    return "Not Found"

def extract_email(text):
    """Extract email using regex."""
    email_match = re.search(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', text)
    return email_match.group(0) if email_match else "Not Found"

def extract_qualification(text):
    """Extract qualifications by matching common degree terms."""
    qualifications = re.findall(r'\b(B(?:\.|achelor)?|M(?:\.|aster)?|Ph\.?D|Diploma|High School|HSC|UG|PG|CS|Engineering|Science)\b', text, re.IGNORECASE)
    return ", ".join(set(qualifications)) if qualifications else "Not Found"

def extract_resume_details_nltk(file_path):
    """Extract details using pdfplumber and NLTK."""
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    
    name = extract_name(text)
    email = extract_email(text)
    qualification = extract_qualification(text)
    
    return {"Name": name, "Qualification": qualification, "Email": email}

# Process all resumes
resume_files = ["Resume01.pdf","Resume02.pdf"]  # List of resume file paths
resume_data = []

for file in resume_files:
    details = extract_resume_details_nltk(file)
    resume_data.append(details)

# Convert to DataFrame and save as CSV
df = pd.DataFrame(resume_data)
output_path = "extracted_resume_data_nltk.csv"
df.to_csv(output_path, index=False)

print(f"Data saved to {output_path}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Data saved to extracted_resume_data_nltk.csv
